In [2]:
from collections import defaultdict
from IPython.display import display
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk.stem.wordnet as wordnet
import numpy as np
import pandas as pd
import sklearn
import string
import time

Requires grants_gov.csv and research_grant_history.csv

In [3]:
sw = set(stopwords.words('english'))
wnl = wordnet.WordNetLemmatizer()

In [4]:
pd.set_option('max_columns', 40)
grants_df = pd.read_csv('../scraped_data/grants_gov.csv', delimiter="~")
grants_df.head()

,OpportunityID,OpportunityTitle,OpportunityNumber,OpportunityCategory,FundingInstrumentType,CategoryOfFundingActivity,CategoryExplanation,CFDANumbers,EligibleApplicants,AdditionalInformationOnEligibility,AgencyCode,AgencyName,PostDate,CloseDate,LastUpdatedDate,AwardCeiling,AwardFloor,EstimatedTotalProgramFunding,ExpectedNumberOfAwards,Description,Version,CostSharingOrMatchingRequirement,ArchiveDate,GrantorContactEmail,GrantorContactEmailDescription,GrantorContactText,AdditionalInformationURL,AdditionalInformationText,OpportunityCategoryExplanation,CloseDateExplanation
0,262148,Establishment of the Edmund S. Muskie Graduate...,SCAPPD-14-AW-161-SCA-08152014,D,CA,O,Public Diplomacy,19.040,25,Eligibility for U.S. institutions is limited t...,DOS-SA,Bureau of South and Central Asian Affairs,8152014,9042014.0,8202014,600000.0,400000.0,600000.0,1.0,The Office of Press and Public Diplomacy of th...,Synopsis 2,No,10042014.0,DelageEM@state.gov,Inquiries,"Ellen Delage, Bureau of South and Central Asia...",NaN,NaN,NaN,NaN
1,262149,Eradication of Yellow Crazy Ants on Johnston A...,F14AS00402,D,CA,"AG,ENV,NR",NaN,15.608,99,The recipient has already been selected for th...,DOI-FWS,Fish and Wildlife Service,8152014,8222014.0,8152014,0.0,0.0,0.0,NaN,Funds under this award are to be used for the ...,Synopsis 1,No,8232014.0,Connie_Sauer@fws.gov,Connie_Sauer@fws.gov,"Connie Sauer, 503-490-0827&lt;br/&gt;Connie_Sa...",http://www.grants.gov/,http://www.grants.gov/,NaN,NaN
2,131073,"Cooperative Ecosystem Studies Unit, Piedmont S...",G12AS20003,D,CA,ST,NaN,15.808,25,This financial assistance opportunity is being...,DOI-USGS1,Geological Survey,11172011,11292011.0,11282011,0.0,0.0,31900.0,1.0,The USGS Southeast Ecological Science Center s...,Synopsis 2,No,12172011.0,fgraves@usgs.gov,fgraves@usgs.gov,"Faith Graves, 703-648-7356&lt;br/&gt;fgraves@u...",http://www.grants.gov/,http://www.grants.gov/,NaN,NaN
3,196613,OVW FY 2012 Training and Technical Assistance ...,OVW-2012-3380,D,CA,LJL,NaN,16.526,"06,12",NaN,USDOJ-OJP-OVW,Office on Violence Against Women,8292012,10152012.0,8292012,2500000.0,NaN,2500000.0,1.0,There are different statutory and regulatory r...,Synopsis 1,No,11142012.0,Christina.Murray@usdoj.gov,Christina.Murray@usdoj.gov,Christina Murray &lt;br/&gt;Phone 202-307-0344,http://www.ovw.usdoj.gov/docs/grantee-reportin...,OVW FY 2012 Training and Technical Assistance ...,CategoryExplanation,NaN
4,65553,Affordable Care Act (ACA): Childhood Obesity R...,RFA-DP-11-007,D,CA,HL,NaN,93.535,99,NaN,HHS-CDC-HHSCDCERA,Centers for Disease Control and Prevention - ERA,1192011,4252011.0,4152011,5250000.0,0.0,NaN,4.0,This Funding Opportunity Announcement (FOA) so...,Synopsis 10,No,5182011.0,pgotim@cdc.gov,General Email,Centers for Disease Control and Prevention (CD...,Please click on Full Announcement at the top o...,Please click on Full Announcement at the top o...,NaN,On-time submission requires that electronic ap...


In [7]:
faculty_df = pd.read_csv('../scraped_data/faculty_profiles.csv', delimiter="~")
faculty_df.head()

,faculty_name,faculty_profile_url,l_expertise,department,title_name,faculty_site_url,lab_url,faculty_email,description,description_links,article_date_1,title_of_news_1,link_to_news_1,description_teaser_1,article_date_2,title_of_news_2,link_to_news_2,description_teaser_2,article_date_3,title_of_news_3,link_to_news_3,description_teaser_3,article_date_4,title_of_news_4,link_to_news_4,description_teaser_4,article_date_5,title_of_news_5,link_to_news_5,description_teaser_5
0,David A. Aaker,/faculty/david-aaker,"business,marketing,branding",Haas School of Business,Professor of Marketing and Public Policy,http://www.haas.berkeley.edu/faculty/aaker.html,http://groups.haas.berkeley.edu/marketing/,aaker@haas.berkeley.edu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Pieter Abbeel,/faculty/pieter-abbeel,"robotics,machine learning",Division of Computer Science/EECS,Professor,http://www.cs.berkeley.edu/~pabbeel,NaN,pabbeel@cs.berkeley.edu,Robotics and Machine Learning.,NaN,"February 22, 2016",“Deep Learning”: A Giant Step for Robots,/bakarfellows/profile/pieter_abbeel,Bakar Fellow Pieter Abbeel studies deep lear...,"February 19, 2016",Three young faculty members honored by White ...,/news/three-young-faculty-members-honored-whit...,Three UC Berkeley faculty members named as r...,"May 21, 2015",New ‘deep learning’ technique enables robot m...,/news/new-deep-learning-technique-enables-robo...,UC Berkeley researchers have developed algor...,"December 17, 2012",Big NSF grant funds research into training ro...,/news/big-nsf-grant-funds-research-training-ro...,"What if robots and humans, working together,...","August 23, 2011",UC Berkeley robotics expert named among world...,/news/uc-berkeley-robotics-expert-named-among-...,"Pieter Abbeel, a UC Berkeley, professor know..."
2,Elizabeth Abel,/faculty/elizabeth-abel,"feminist theory,psychoanalysis,Virginia Woolf,...",Department of English,Professor of English,http://english.berkeley.edu/profiles/5,NaN,eabel@uclink.berkeley.edu,Elizabeth Abel's general research interest is...,NaN,"February 15, 2011","Jim Crow signs as symbols of subjugation, tro...",/news/jim-crow-signs-symbols-subjugation-troph...,"In the mid 1960s, landmark laws brought an o...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Dor Abrahamson,/faculty/dor-abrahamson,"mathematical cognition,design-based research,m...",Graduate School of Education,Associate Professor of Cognition and Development,http://gse.berkeley.edu/people/dor-abrahamson,http://edrl.berkeley.edu/,dor@berkeley.edu,Dor Abrahamson studies the process of mathema...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Norman Abrahamson,/faculty/norman-abrahamson,"civil and environmental engineering,earthquake...",Department of Civil and Environmental Engineering,Adjunct Professor of Civil and Environmental E...,http://www.ce.berkeley.edu/faculty/faculty.php...,NaN,naa3@earthlink.net,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
grant_history_df = pd.read_csv('../scraped_data/research_grant_history.csv', delimiter=",")
grant_history_df.head()

/home/jerry/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Activity Type,Amount,Sponsor Class,Sponsor,Division,Department,Fund,UCB Award Number,PI Name,Project Begin Date,Project End Date,Title,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
0,Applied research,"$179,032",State of California,California Department of Health Care Services,School of Public Health,NaN,15952.0,021331-002,"Colford Jr, John M",7/1/2006,9/30/2006,DNS AIDS Training,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Basic research,"$154,578",State of California,California Department of Social Services,School of Social Welfare,NaN,15959.0,021362-002,"Needell, Barbara",7/1/2006,9/30/2006,Performance Indicators/California Children's S...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Instruction,"$225,000",State of California,California Department of Social Services,School of Social Welfare,Social Welfare,15960.0,021363-002,"Midgley, James",7/1/2006,9/30/2006,Title IV-E Social Work Training Program,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Instruction,"$47,138",State of California,California Department of Social Services,School of Social Welfare,Social Welfare,15960.0,021363-002,"Midgley, James",7/1/2006,9/30/2006,Title IV-E Social Work Training Program,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Basic research,"$65,000",Federal,NIH National Institutes of Health - Miscellaneous,VC Res Other Research Units,The California Institute for Quantitative Bios...,78561.0,021425-002,"Keasling, Jay",7/1/2006,12/31/2006,Model-Driven Strain Engineering for Isoprenoid...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
nsf_grants_df = pd.read_csv('../scraped_data/nsf_grants.csv', delimiter="~")
nsf_grants_df.head()

,Headline,Due Date Start,Due Date End,Description,Link
0,Resource Implementations for Data Intensive Re...,None,2017-02-27,As part of NSF’s Cyberinfrastructure Framework...,https://nsf.gov/funding/pgm_summ.jsp?pims_id=5...
1,Computer Science for All,None,2017-02-28,This program aims to provide all U.S.students ...,https://nsf.gov/funding/pgm_summ.jsp?pims_id=5...
2,EPSCoR Research Infrastructure Improvement Tra...,None,2017-02-28,The Experimental Program to Stimulate Competit...,https://nsf.gov/funding/pgm_summ.jsp?pims_id=5...
3,Industry-University Cooperative Research Cente...,None,2017-02-28,The Industry-University Cooperative Research C...,https://nsf.gov/funding/pgm_summ.jsp?pims_id=5...
4,SOCIOLOGY PROGRAM - Doctoral Dissertation Rese...,None,2017-02-28,The Sociology Program supports basic research ...,https://nsf.gov/funding/pgm_summ.jsp?pims_id=5...


In [9]:
descriptions = grants_df['Description']

In [10]:
cleaned = []
for desc in descriptions:
    try:
        desc = desc.lower()
        desc = desc.translate(str.maketrans('','',string.punctuation))
        desc = desc.translate(str.maketrans('','','1234567890'))
        first = filter(lambda x: x.lower() not in sw and len(x) > 3, desc.split())
        second = [wnl.lemmatize(x) for x in first]
        cleaned.append(second)
    except:
        cleaned.append(["Nothing"])
    

In [35]:
print(cleaned[0])

['part', 'nsf’s', 'cyberinfrastructure', 'framework', 'century', 'science', 'engineering', 'activity', 'directorate', 'social', 'behavioral', 'economic', 'science', 'seek', 'develop', 'userfriendly', 'largescale', 'nextgeneration', 'data', 'resource', 'relevant', 'analytic', 'technique', 'advance', 'fundamental', 'research', 'area', 'study', 'successful', 'proposal', 'within', 'financial', 'resource', 'provided', 'award', 'construct', 'database', 'andor', 'relevant', 'analytic', 'technique', 'produce', 'finished', 'product', 'enable', 'type', 'dataintensive', 'research', 'database', 'technique', 'significant', 'impact', 'either', 'across', 'multiple', 'field', 'within', 'broad', 'disciplinary', 'area', 'enabling', 'type', 'dataintensive', 'research', 'science']


In [13]:
corpus = [' '.join(doc) for doc in cleaned]

In [14]:
grants_vectorizer = TfidfVectorizer(min_df=4)
grants_matrix = grants_vectorizer.fit_transform(corpus)

In [37]:
print(grants_matrix.shape)
print(len(grants_vectorizer.vocabulary_))

(41536, 19811)
19811


In [27]:
# Prepare NSF grants
cleaned = []
nsf_descriptions = nsf_grants_df['Description']
    
for desc in nsf_descriptions:
    try:
        desc = desc.lower()
        desc = desc.translate(str.maketrans('','',string.punctuation))
        desc = desc.translate(str.maketrans('','','1234567890'))
        first = filter(lambda x: x.lower() not in sw and len(x) > 3, desc.split())
        second = [wnl.lemmatize(x) for x in first]
        cleaned.append(second)
    except:
        cleaned.append(["Nothing"])
        
nsf_corpus = [' '.join(doc) for doc in cleaned]

In [28]:
nsf_vectorizer = TfidfVectorizer(min_df=4, vocabulary=grants_vectorizer.vocabulary_)
nsf_matrix = nsf_vectorizer.fit_transform(nsf_corpus)

In [29]:
nsf_matrix.shape

(484, 19811)

Creates faculty profiles. Currently takes all past research from research_grant_history.csv and vectorizes using vocabulary from grants_gov.csv and takes the mean (possibly not the best way to do this).

In [16]:
faculty_profiles = defaultdict(list)
for index, row in grant_history_df.iterrows():
    faculty_profiles[row["PI Name"]].append(row["Title"])

In [17]:
faculty_vectorizer = TfidfVectorizer(vocabulary=grants_vectorizer.vocabulary_)
for faculty in faculty_profiles:
    research = faculty_profiles[faculty]
    vectorized = faculty_vectorizer.fit_transform(research)
    faculty_profiles[faculty] = np.mean(vectorized, axis=0)

In [18]:
faculty_list = list(faculty_profiles.keys())

# Some sanity checks
print(faculty_profiles[faculty_list[1]].shape)
print(grants_matrix[0][0].shape)

(1, 19811)
(1, 19811)


In [19]:
top_matches = {}
for i in range(10):
    faculty = faculty_list[i]
    profile = faculty_profiles[faculty]
    matches = []
    for j in range(grants_matrix.shape[0]):
        grant = grants_matrix[j]
        matches.append((np.dot(grant.toarray(), profile.T), j))
    top_matches[faculty] = [x[1] for x in sorted(matches)[-5:]]


In [22]:
print(top_matches)

{'Colford Jr, John M': [10284, 22738, 9579, 19965, 6515], 'Needell, Barbara': [23976, 18247, 5838, 36555, 16228], 'Midgley, James': [20410, 37366, 32041, 32039, 33531], 'Keasling, Jay': [27970, 19374, 19763, 28361, 26823], 'Carmichael, A Christopher': [10660, 14171, 25589, 9538, 29854], 'Nadeau, Robert': [31286, 9191, 17127, 15087, 35325], 'Welter, Stephen C': [10508, 22425, 30228, 15258, 29752], 'Liphardt, Jan T': [34471, 36432, 8809, 16274, 30515], 'Fairfax, Sally K': [39942, 21422, 35080, 3507, 31333], 'Alber, Thomas C': [16495, 3893, 6900, 13947, 14163]}


In [23]:
matches = defaultdict(list)
for faculty in top_matches:
    for index in top_matches[faculty]:
        matches[faculty].append(grants_df[index:index+1])

In [24]:
# Grant matches for Colford Jr, John M from grants_gov
for match in matches['Colford Jr, John M']:
    display(match)

,OpportunityID,OpportunityTitle,OpportunityNumber,OpportunityCategory,FundingInstrumentType,CategoryOfFundingActivity,CategoryExplanation,CFDANumbers,EligibleApplicants,AdditionalInformationOnEligibility,AgencyCode,AgencyName,PostDate,CloseDate,LastUpdatedDate,AwardCeiling,AwardFloor,EstimatedTotalProgramFunding,ExpectedNumberOfAwards,Description,Version,CostSharingOrMatchingRequirement,ArchiveDate,GrantorContactEmail,GrantorContactEmailDescription,GrantorContactText,AdditionalInformationURL,AdditionalInformationText,OpportunityCategoryExplanation,CloseDateExplanation
10284,281398,Strengthening Applied Epidemiology and Sustain...,CDC-RFA-GH15-161902CONT16,C,CA,HL,NaN,93.318,25,This award will be a continuation of funds int...,HHS-CDC,Centers for Disease Control and Prevention,2092016,6102016.0,5272016,0.0,0.0,0.0,1.0,Strengthening Applied Epidemiology and Sustain...,Synopsis 5,No,7012016.0,ogstims@cdc.gov,ogstims@cdc.gov,Office of Grant Services (OGS) &lt;br/&gt;Tech...,NaN,NaN,NaN,NaN


,OpportunityID,OpportunityTitle,OpportunityNumber,OpportunityCategory,FundingInstrumentType,CategoryOfFundingActivity,CategoryExplanation,CFDANumbers,EligibleApplicants,AdditionalInformationOnEligibility,AgencyCode,AgencyName,PostDate,CloseDate,LastUpdatedDate,AwardCeiling,AwardFloor,EstimatedTotalProgramFunding,ExpectedNumberOfAwards,Description,Version,CostSharingOrMatchingRequirement,ArchiveDate,GrantorContactEmail,GrantorContactEmailDescription,GrantorContactText,AdditionalInformationURL,AdditionalInformationText,OpportunityCategoryExplanation,CloseDateExplanation
22738,41308,Water Supply and Sanitation Activities,RFA-511-08-010,D,CA,O,Water and Sanitation Activities,98.001,99,NaN,USAID-BOL,Bolivia USAID-La Paz,4172008,5142008.0,4172008,300000.0,50000.0,1000000.0,4.0,USAID/Bolivia intends to continue to support a...,Synopsis 1,Yes,6132008.0,jzenteno@usaid.gov,jzenteno@usaid.gov,Jose Zenteno &lt;br/&gt;Acquisition Specialist...,NaN,NaN,NaN,NaN


,OpportunityID,OpportunityTitle,OpportunityNumber,OpportunityCategory,FundingInstrumentType,CategoryOfFundingActivity,CategoryExplanation,CFDANumbers,EligibleApplicants,AdditionalInformationOnEligibility,AgencyCode,AgencyName,PostDate,CloseDate,LastUpdatedDate,AwardCeiling,AwardFloor,EstimatedTotalProgramFunding,ExpectedNumberOfAwards,Description,Version,CostSharingOrMatchingRequirement,ArchiveDate,GrantorContactEmail,GrantorContactEmailDescription,GrantorContactText,AdditionalInformationURL,AdditionalInformationText,OpportunityCategoryExplanation,CloseDateExplanation
9579,280588,Conducting Public Health Research through Fiel...,RFA-GH-11-00105CONT16,C,CA,HL,NaN,93.283,25,This award will be a continuation of funds int...,HHS-CDC-HHSCDCERA,Centers for Disease Control and Prevention - ERA,12172015,2172016.0,12172015,0.0,0.0,0.0,2.0,Conducting Public Health Research through Fiel...,Synopsis 1,No,3182016.0,pgotim@cdc.gov,pgotim@cdc.gov,Centers for Disease Control and Prevention (CD...,NaN,NaN,NaN,NaN


,OpportunityID,OpportunityTitle,OpportunityNumber,OpportunityCategory,FundingInstrumentType,CategoryOfFundingActivity,CategoryExplanation,CFDANumbers,EligibleApplicants,AdditionalInformationOnEligibility,AgencyCode,AgencyName,PostDate,CloseDate,LastUpdatedDate,AwardCeiling,AwardFloor,EstimatedTotalProgramFunding,ExpectedNumberOfAwards,Description,Version,CostSharingOrMatchingRequirement,ArchiveDate,GrantorContactEmail,GrantorContactEmailDescription,GrantorContactText,AdditionalInformationURL,AdditionalInformationText,OpportunityCategoryExplanation,CloseDateExplanation
19965,100914,Judith Mountains Acid Drainage Characterization,L11AS00137,D,CA,NR,NaN,15.236,25,This financial assistance opportunity is ...,DOI-BLM,Bureau of Land Management,6232011,7242011.0,7132011,35000.0,2500.0,90000.0,1.0,Study of water quality in the Judith Mountains.,Synopsis 3,No,8012011.0,landerso@blm.gov,GMO,"Lori Anderson, Grants Management Officer, 406-...",http://www.grants.gov,http://www.grants.gov,NaN,NaN


,OpportunityID,OpportunityTitle,OpportunityNumber,OpportunityCategory,FundingInstrumentType,CategoryOfFundingActivity,CategoryExplanation,CFDANumbers,EligibleApplicants,AdditionalInformationOnEligibility,AgencyCode,AgencyName,PostDate,CloseDate,LastUpdatedDate,AwardCeiling,AwardFloor,EstimatedTotalProgramFunding,ExpectedNumberOfAwards,Description,Version,CostSharingOrMatchingRequirement,ArchiveDate,GrantorContactEmail,GrantorContactEmailDescription,GrantorContactText,AdditionalInformationURL,AdditionalInformationText,OpportunityCategoryExplanation,CloseDateExplanation
6515,277321,USAID/Uganda Sanitation Program,RFA-617-15-000008,D,CA,HL,NaN,98.001,25,The United States Agency for International Dev...,USAID-UGA,Uganda USAID-Kampala,6192015,7212015.0,7212015,0.0,0.0,0.0,1.0,USAID/Uganda anticipates awarding a 5-year Agr...,Synopsis 2,Yes,7312015.0,KampalaUSAIDSolicita@USAID.gov,KampalaUSAIDSolicita@USAID.gov,Alimo Florence &lt;br/&gt;Acquisition and Assi...,NaN,NaN,CategoryExplanation,NaN


In [32]:
nsf_top_matches = {}
for i in range(10):
    faculty = faculty_list[i]
    profile = faculty_profiles[faculty]
    matches = []
    for j in range(nsf_matrix.shape[0]):
        grant = nsf_matrix[j]
        matches.append((np.dot(grant.toarray(), profile.T), j))
    nsf_top_matches[faculty] = [x[1] for x in sorted(matches)[-5:]]
    
nsf_matches = defaultdict(list)
for faculty in nsf_top_matches:
    for index in nsf_top_matches[faculty]:
        nsf_matches[faculty].append(nsf_grants_df[index:index+1])

In [34]:
# Grant matches for Colford Jr, John M from NSF grants
for match in nsf_matches['Colford Jr, John M']:
    display(match)

,Headline,Due Date Start,Due Date End,Description,Link
332,Advancing Health Services through System Model...,Not listed,Not listed,"NSF, in collaboration with the Health Informat...",https://nsf.gov/funding/pgm_summ.jsp?pims_id=5...


,Headline,Due Date Start,Due Date End,Description,Link
126,Chemical Catalysis,2017-09-01,2017-10-02,"""Note: For proposals with significant emphasis...",https://nsf.gov/funding/pgm_summ.jsp?pims_id=5...


,Headline,Due Date Start,Due Date End,Description,Link
182,Chemistry of Life Processes,2017-10-01,2017-10-31,"""Note: for a proposal with significant emphasi...",https://nsf.gov/funding/pgm_summ.jsp?pims_id=5...


,Headline,Due Date Start,Due Date End,Description,Link
181,Chemical Measurement and Imaging,2017-10-01,2017-10-31,Note: For proposals with significant emphasis ...,https://nsf.gov/funding/pgm_summ.jsp?pims_id=5...


,Headline,Due Date Start,Due Date End,Description,Link
316,Hydrologic Sciences (HS),Not listed,Not listed,The Hydrologic Sciences Program focuses on the...,https://nsf.gov/funding/pgm_summ.jsp?pims_id=1...


In [50]:
# Links to the matches
pd.options.display.max_colwidth = 100
for match in nsf_matches['Colford Jr, John M']:
    print(match['Link'].to_string())

332    https://nsf.gov/funding/pgm_summ.jsp?pims_id=504720&org=NSF&sel_org=NSF&from=fund
126    https://nsf.gov/funding/pgm_summ.jsp?pims_id=503418&org=NSF&sel_org=NSF&from=fund
182    https://nsf.gov/funding/pgm_summ.jsp?pims_id=503417&org=NSF&sel_org=NSF&from=fund
181    https://nsf.gov/funding/pgm_summ.jsp?pims_id=503413&org=NSF&sel_org=NSF&from=fund
316    https://nsf.gov/funding/pgm_summ.jsp?pims_id=13684&org=NSF&sel_org=NSF&from=fund
